# Fruit Ripeness: Unripe, Ripe, and Rotten Image classification

# Install required packages (no kagglehub needed)

In [ ]:
!pip install tensorflow numpy matplotlib

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import matplotlib.pyplot as plt


## 1. Data & Model prep


In [16]:
# Hyper params - OPTIMIZED FOR SPEED
BATCH_SIZE = 128  # Increased from 20 for faster processing
IMAGE_SIZE = (96, 96)  # MobileNetV2 optimal size - faster than 224x224
EPOCHS = 12

# Using local Fruit Ripeness dataset - CORRECTED PATHS
DATA_DIR = './Fruit Ripeness: Unripe, Ripe, and Rotten/fruit_ripeness_dataset/dataset'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'dataset', 'train')  # Use nested dataset/train as validation
TEST_DIR = os.path.join(DATA_DIR, 'test')

In [17]:
# load the data with ImageDataGenerator to load images , resize them, and apply basic data augmentation(rotaiton, flips...) to improve the model's robustness.
# Rescale to [0, 1]
train_datagen = ImageDataGenerator(
    rescale = 1./255 ,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
# no augmentation for validaiton
valid_datagen = ImageDataGenerator(
    rescale = 1./255
)
# load the training data
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    class_mode= 'categorical',
    shuffle=True
)
validation_generator = valid_datagen.flow_from_directory(
    VALID_DIR,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    class_mode= 'categorical',
    shuffle=False
)
# the number of classes for the final layer
NUM_CLASSES = train_generator.num_classes
print(f"Total classes detected : {NUM_CLASSES}")

Found 16217 images belonging to 9 classes.
Found 16217 images belonging to 9 classes.
Total classes detected : 9


![img](https://encrypted-tbn3.gstatic.com/licensed-image?q=tbn:ANd9GcS8ZAQqtM-09H9jSR8hOrkmPZkc9c72vG4q97zfwxLmV5101IvOKMpveIKsUGEGooWe-VT6HqSqqps5EPS0vxdXeJ5tckxYrQwiIAtTxLSFUG_rcwE)

In [18]:
# Load base model
# Load MobileNetV2 pre-trained on ImageNet, without the top classification layer
base_model = tf.keras.applications.MobileNetV2(
    input_shape = IMAGE_SIZE + (3,),
    include_top = False,
    weights = 'imagenet'
)
# Freeze the base model to prevent weights form being updated during the training
base_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [19]:
# Build the custom classififer Head
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation = 'relu'),
    Dropout(0.2),# regularization to prevent overfitting
    Dense(NUM_CLASSES, activation = 'softmax') # final classification layer
])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_96             │ (None, 3, 3, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,423,113 (9.24 MB)

 Trainable params: 165,129 (645.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [20]:
model.compile(
    optimizer = Adam(learning_rate = 0.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

# 2. Training

In [21]:
history = model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data = validation_generator
)

Epoch 1/12
127/127 ━━━━━━━━━━━━━━━━━━━━ 112s 869ms/step - accuracy: 0.5473 - loss: 1.2788 - val_accuracy: 0.7959 - val_loss: 0.6188
Epoch 2/12
127/127 ━━━━━━━━━━━━━━━━━━━━ 97s 765ms/step - accuracy: 0.7821 - loss: 0.6061 - val_accuracy: 0.8573 - val_loss: 0.4084
Epoch 3/12
127/127 ━━━━━━━━━━━━━━━━━━━━ 95s 749ms/step - accuracy: 0.8255 - loss: 0.4727 - val_accuracy: 0.8811 - val_loss: 0.3361
Epoch 4/12
127/127 ━━━━━━━━━━━━━━━━━━━━ 90s 713ms/step - accuracy: 0.8570 - loss: 0.3969 - val_accuracy: 0.8923 - val_loss: 0.2993
Epoch 5/12
127/127 ━━━━━━━━━━━━━━━━━━━━ 90s 708ms/step - accuracy: 0.8667 - loss: 0.3638 - val_accuracy: 0.9021 - val_loss: 0.2684
Epoch 6/12
127/127 ━━━━━━━━━━━━━━━━━━━━ 90s 708ms/step - accuracy: 0.8796 - loss: 0.3288 - val_accuracy: 0.9076 - val_loss: 0.2507
Epoch 7/12
127/127 ━━━━━━━━━━━━━━━━━━━━ 90s 708ms/step - accuracy: 0.8876 - loss: 0.3107 - val_accuracy: 0.9129 - val_loss: 0.2355
Epoch 8/12
127/127 ━━━━━━━━━━━━━━━━━━━━ 89s 705ms/step - accuracy: 0.8908 - loss: 

In [24]:
# save the trained keras model for potential future use
model.save('ripness_cnn_model.h5')

# 3. Plotting results

In [25]:
# Plot training history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(NUM_EPOCHS)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

NameError: name 'NUM_EPOCHS' is not defined

# 4. Convert the Keras model to TFLite

In [26]:
# Initialize the TFLite converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply default optimization (Post-Training Quantization) for smaller size and faster inference
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model file
tflite_model_path = 'ripeness_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved to: {tflite_model_path}")

INFO:tensorflow:Assets written to: /tmp/tmpwleqceti/assets


INFO:tensorflow:Assets written to: /tmp/tmpwleqceti/assets


Saved artifact at '/tmp/tmpwleqceti'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='keras_tensor_482')
Output Type:
  TensorSpec(shape=(None, 9), dtype=tf.float32, name=None)
Captures:
  138022405282448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138022405283984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138022405283792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138022405283216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138022405284560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138022405282640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138022405284176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138022405284368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138022405283408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138022405285520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138022405284

W0000 00:00:1764959738.612156  203491 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1764959738.612171  203491 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-05 19:35:38.612398: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpwleqceti
2025-12-05 19:35:38.619544: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-12-05 19:35:38.619550: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpwleqceti
I0000 00:00:1764959738.669075  203491 mlir_graph_optimization_pass.cc:437] MLIR V1 optimization pass is not enabled
2025-12-05 19:35:38.682325: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-12-05 19:35:39.038039: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpwleqceti
2025-12-05 19:35:39.131317: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 

# 5. Save the Label map
since the flutter pap needs a lsit f the class names in the correct order to interpret the model's output

In [27]:
# Get class indices and map them to class names
labels = sorted(train_generator.class_indices.items(), key=lambda x: x[1])
class_names = [name for name, index in labels]

# Save class names to a text file
labels_file_path = 'ripeness_labels.txt'
with open(labels_file_path, 'w') as f:
    f.write('\n'.join(class_names))

print(f"Label map saved to: {labels_file_path}")
print("Final Classes:", class_names)

Label map saved to: ripeness_labels.txt
Final Classes: ['freshapples', 'freshbanana', 'freshoranges', 'rottenapples', 'rottenbanana', 'rottenoranges', 'unripe apple', 'unripe banana', 'unripe orange']
